In [3]:
import pandas as pd
import re
import contractions

import nltk
nltk.download('wordnet', quiet=True)
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [2]:
!pip install contractions

  Using cached contractions-0.1.72-py2.py3-none-any.whl (8.3 kB)
  Using cached anyascii-0.3.1-py3-none-any.whl (287 kB)


In [5]:
data_path = "../data/"
raw_data = pd.read_csv(data_path + "Tweets.csv")

raw_data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [6]:
def clean_data(x):
    x = x.lower() # convert to all lower case
    # x = re.sub('http\S+', '', x) # remove URLs
    # x = re.sub('<.*?>', '', x) # remove HTML tags
    x = contractions.fix(x) # expand contractions
    x = re.sub('[^a-z]', ' ', x) # remove non-alphabetic characters
    x = re.sub('\s\s+', ' ', x) # remove extra whitespace
    return x

In [7]:
df = raw_data.copy(deep=True)
df.text = df.text.apply(clean_data)
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,virginamerica what dhepburn said,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,virginamerica plus you have added commercials...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,virginamerica i did not today must mean i nee...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,virginamerica it is really aggressive to blas...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,virginamerica and it is a really big bad thin...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [8]:
stopwords_eng = set(stopwords.words('english'))
wnl = WordNetLemmatizer()

remove_stopwords = lambda x: ' '.join([word for word in word_tokenize(x) if word not in stopwords_eng])
lemmatizer = lambda x: ' '.join(
    [
        wnl.lemmatize(
            wnl.lemmatize(
                wnl.lemmatize(
                    wnl.lemmatize(
                        word,
                        pos='n'
                    ),
                pos='v'
                ),
            pos='a'
            ),
        pos='r'
        ) 
        for word in word_tokenize(x)
    ]
)
# def preprocess(x):
#     x = remove_stopwords(x)
#     x = lemmatizer(x)
#     return x

# df.text = df.text.apply(preprocess)
# df.head()

In [9]:
def convert_labels(x):
    x = x.strip()
    if x == "positive":
        return 1
    elif x == "negative":
        return -1
    elif x == "neutral":
        return 0
df.airline_sentiment = df.airline_sentiment.apply(convert_labels)

In [10]:
df.to_csv(data_path + "cleaned_Tweets.csv")

In [11]:
intent = df[df['negativereason'].notna()]
intent.reset_index(drop=True, inplace=True)
intent.to_csv(data_path + "intent_Tweets.csv")